<!DOCTYPE html>
<html>
<head>
    <style>
        .center-image {
            display: block;
            margin: 0 auto;
        }
    </style>
</head>
<body>
    <img src="https://th.bing.com/th/id/OIP.18EZOYy8eAuoeZwXcmCUGQAAAA?rs=1&pid=ImgDetMain" alt="Logo" class="center-image">
</body>
</html>

En este notebook se abordará la solución a un problema. El primer desafío es la carga de datos en un tablero de Power BI, donde la conexión pueda ser directa al RMP. De esta forma, se asegura una carga continua de datos sin problemas de espacio en almacenamiento.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
# Configuro pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_excel("Base de Ventas 10-10 09hs 1.xlsx",sheet_name='BBDD')

In [3]:
df['Canal'].unique()

array(['Retail Especializado', 'Grupo de Compras', 'Hipermercados', 'B2C',
       nan, 'B2B', 'Otros Retails', 'RETAIL Especializado',
       'Retail especializado'], dtype=object)

In [24]:
df_venta_historica = pd.read_excel("Base Historica.xlsx",sheet_name='ventas')

In [26]:
df_venta_historica

,Soc.,Solicitante,ClFac,Clase de factura,Pos.,Solic.,A#o,Fecha factura,CDis,Referencia,N# doc.,UM,Mon.,UM2,OrgVt,Grupo art.,Grupo de art#culos,CeBe,Mon.3,Ctd.facturada,Valor neto,ZPAC,ZPCA,ZPCP,ZPCS,Impuestos Internos,Descuento,Importe del impuesto,Valor bruto,PUNI Cliente,PUNI Lista,Costes internos,Material,Denominaci#n de posici#n,Alm.,Per#odo,Mon.4,Mon.5,Mon.6,Mon.7,GClt,Doc.fact.,T/C,LP,CPag,Se,II,Mon.8,Ce.,Asignaci#n,Creado el,Creado por,Md.P,Descripcion condicion de pago,Doc.venta,Referencia cliente,Mes,Año,Mes-Año,Tipo de Venta,Linea,Linea OEM,Control Ifsa,TC,Valor Neto USD,Marca,Canal,Subcanal,Hijos
0,AR11,MINI PRECIOS S. R. L.,ZVP,Factura Programada,20,1000905,2022.0,2022-01-12,VP,0127A00445194,1.600001e+09,UN,ARS,UN,1010,7,NaN,102001,ARS,5,169747.93,0,0.0,0.0,0.0,11119.83,-169747.93,35647.07,205395.00,33949.59,0,0.0,PNB045372,AAI BC35WFQ,BG15,0,ARS,ARS,ARS,ARS,NaN,90004973,NaN,NaN,D003,LH,NaN,ARS,SP01,315107,44573,OP_LOG_WS,NaN,BGH Pago 7 días Fecha Factura,30000612,PROG VIEJAS,1,2022,1-2022,Programada,AAI,AAI,NaN,105.02,1616.339078,BGH,Grupo de Compras,Red Del Sol S.R.L.,-
1,AR11,CARSA S.A.,ZVP,Factura Programada,10,1000220,2022.0,2022-01-10,VP,0127A00445119,1.600001e+09,UN,ARS,UN,1010,7,NaN,102001,ARS,16,560097.32,0,0.0,0.0,0.0,36690.80,-560097.32,151506.33,711603.65,35006.08,0,0.0,PNB045372,AAI BC35WFQ,BG15,0,ARS,ARS,ARS,ARS,NaN,90004611,NaN,NaN,D005,LH,NaN,ARS,SP01,4501358452,44571,OP_LOG_WS,NaN,BGH Pago 15 días Fecha Factura,30001617,4501358452,1,2022,1-2022,Programada,AAI,AAI,NaN,105.02,5333.244334,BGH,Retail Especializado,CARSA S.A.,-
2,AR11,DANTE TANINO HOGAR S.A.,ZVP,Factura Programada,20,1000339,2022.0,2022-01-12,VP,0127A00445196,1.600001e+09,UN,ARS,UN,1010,7,NaN,102001,ARS,10,339495.87,0,0.0,0.0,0.0,22239.66,-339495.87,71294.13,410790.00,33949.59,0,0.0,PNB045372,AAI BC35WFQ,BG15,0,ARS,ARS,ARS,ARS,NaN,90004975,NaN,NaN,D003,LH,NaN,ARS,SP01,315126,44573,OP_LOG_WS,NaN,BGH Pago 7 días Fecha Factura,30000617,315126,1,2022,1-2022,Programada,AAI,AAI,NaN,105.02,3232.678252,BGH,Otros Retails,DANTE TANINO HOGAR S.A.,-
3,AR11,FRAVEGA S.A.C.I.E.I.,ZRE4,NC Rechazo Cliente (OR),10,1000095,2022.0,2022-01-31,VP,0127A00097963,2.600000e+09,UN,ARS,UN,1010,7,NaN,102001,ARS,-1,-32644.63,0,0.0,0.0,0.0,-2138.48,32644.63,-7716.54,40361.17,32644.63,0,31665.9,PNB045372,AAI BC35WFQ,BG06,0,ARS,ARS,ARS,ARS,NaN,90008157,NaN,NaN,D005,LH,NaN,ARS,SP01,90008157,44592,OECHAVARRIA,NaN,BGH Pago 15 días Fecha Factura,60000214,01A012700443366,1,2022,1-2022,Spot,AAI,AAI,NaN,105.02,-310.842030,BGH,Retail Especializado,FRAVEGA S.A.C.I.E.I.,-
4,AR11,MONTAÑA ARIEL AMIN,ZVP,Factura Programada,30,1000517,2022.0,2022-01-17,VP,0127A00445309,1.600002e+09,UN,ARS,UN,1010,7,NaN,102001,ARS,5,169747.93,0,0.0,0.0,0.0,11119.83,-169747.93,46680.69,216428.62,33949.59,0,0.0,PNB045372,AAI BC35WFQ,BG15,0,ARS,ARS,ARS,ARS,NaN,90005729,NaN,NaN,D003,LH,NaN,ARS,SP01,315071,44578,OP_LOG_WS,NaN,BGH Pago 7 días Fecha Factura,30000602,OTROS AAI V NOV-DIC,1,2022,1-2022,Programada,AAI,AAI,NaN,105.02,1616.339078,BGH,Otros Retails,MONTAÑA ARIEL AMIN,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182983,AR11,SERVITEC 9 DE JULIO SA,ZVP,Factura Programada,60,1001596,2025.0,2024-08-30,VP,0127A00483353,1.600040e+09,UN,ARS,UN,1010,7,NaN,102002,ARS,9,9691390.71,0,0.0,0.0,0.0,920663.26,-9691390.71,2810503.31,12501894.02,1076821.19,0,0.0,AR24BSHQAWK2BG,AAI AR24BSHQAWK2BG - SAMSUNG,NaN,0,ARS,ARS,ARS,ARS,NaN,90198266,NaN,NaN,D010,LH,NaN,ARS,SP01,30007794,45534,OP_LOG_WS,NaN,BGH Pago 60 días Fecha Factura,191347,Prog AAI Red Fiel Ag,8,2024,8-2024,Programada,AAI,AAI,NaN,953.50,10164.017525,Samsung,Grupo de Compras,Integracion Hogar,-
182984,AR11,SERVITEC 9 DE JULIO SA,ZVP,Factura Programada,63,1001596,2025.0,2024-08-30,VP,0127A00483353,1.600040e+09,UN,ARS,UN,101

In [ ]:
# Conectar a la base de datos BGH
conexion_config = {
    'user': 'root',
    'password': 'Ayla2022',
    'host': 'localhost',
    'port': 3306,
    'database': 'BGH'
}

# Crear el motor de SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{conexion_config['user']}:{conexion_config['password']}@{conexion_config['host']}:{conexion_config['port']}/{conexion_config['database']}")

# Función para subir un DataFrame en partes
def subir_df_por_partes(df, nombre_tabla, engine, partes=5):
    try:
        # Limpiar los nombres de columnas eliminando espacios en blanco
        df.columns = df.columns.str.strip()

        # Dividir el DataFrame en 'partes' partes
        dfs = np.array_split(df, partes)

        # Subir cada parte del DataFrame
        for i, df_part in enumerate(dfs):
            print(f"Subiendo parte {i+1} de {partes}...")
            df_part.to_sql(nombre_tabla, con=engine, if_exists='append', index=False)
        
        print(f"Tabla '{nombre_tabla}' subida correctamente en {partes} partes.")

    except Exception as e:
        print(f"Error al subir la tabla '{nombre_tabla}': {e}")
        # Intentar rollback manual
        try:
            engine.raw_connection().rollback()
            print("Rollback realizado manualmente.")
        except Exception as rollback_error:
            print(f"Error al intentar hacer rollback: {rollback_error}")

        engine.dispose()
        return False

    return True

# Dividir el DataFrame 'df_venta_historica' en 5 partes y subirlo
nombre_tabla = 'base_ventas_historica'
subir_df_por_partes(df_venta_historica, nombre_tabla, engine, partes=5)

# Cerrar la conexión del engine
engine.dispose()


c:\Users\sosaj\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Subiendo parte 1 de 5...
Subiendo parte 2 de 5...
Subiendo parte 3 de 5...
Subiendo parte 4 de 5...
Subiendo parte 5 de 5...
Tabla 'base_ventas_historica' subida correctamente en 5 partes.


In [3]:
df

,Creado el,Hora,Doc.venta,BE,Texto:Estado de bloqueo de entrega total,StTo,Texto:Status total de las verificaciones,EG,Texto:Estado global de procesamiento (ca,Creado por,Fecha doc.,OrgVt,Organización de ventas,CDis,Mon.,Se,Tipo doc.,Tipo de documento comercial,ClVt,Clase de documento de ventas,GClt,Grupo de clientes,ZV,Zona de ventas,CPag,Condición de pago - desc,Texto:Motivo de pedido (motivo de la ope,GVen,OfVta,Nombre Vendedor,Referencia cliente,Solic.,Solicitante,Pos.,Material,Número de material,Código EAN/UPC,Grupo art.,Grupo de artículos,ValorNeto,Mon..1,UM,Unidad de medida de venta,Ce.,Alm.,Almacén,CeBe,Fij,Rep.,SEn,Fe.entrega,Hora.1,Precio Unitario,Mon..2,Entrega,REMITO,Factura,NRO Legal,Fecha Fectura,MoPed,Div.,Jquía.productos,FAMILIA,TPos,Válido de,Validez a,CL,Cluster,SC,Sub Canal,Grupo precio cliente,Grupo de precio de cliente,FeCrEntreg,Rz,Estado de rechazo (todas las posiciones),Cantidad pedido,UM.1,Ctd.confirmada,UM.2,Cantidad entrg.,UMB,Cant. Fact,UM.3,Ctd.conf.pend,UMB.1,Ctd.pendiente,UMB.2,Cantidad en Back Order,UM.4,Back Order por créditos,UM.5,Cantidad en Back Order C o P,UM.6,Valor Factura,Mon..3,Cant. Conf. Hijo 1,UM.7,Cant. Conf. Hijo 2,UM.8,Hijos confirmados desparejos,UM.9,Destinatario,DEST CALLE,Dest Región,Transportista,Transportista.1,Nombre 4,Pl.transp.,MR,Texto:Motivo de rechazo de documentos de,Operador Logistico,Fecha de Seteo Op. Log.,Usuario de Seteo Op. Log.,Fecha de Seteo Op. Log..1,EstModifClte,TipR,ZPAC,Mon..4,ZPCA,Mon.2,ZPCP,Mon.3,ZPCS,Mon.4,ZPIN,Mon.5,Md.P,Modalidad de Pago,Fecha factura,Tipo de venta,Línea,Línea OEM,AA SUB,AA,Retención,Tipo de pedido,Con Stock,Pedido final,Clasif cliente,Clasif Tablero,Tablero SI,DESC ALMACEN,MES,AÑO Pend,fecha nueva entrega,MES FACT,AÑO FACT,Valor Asignados,suma facturado + asignado,Canal,Para Status Precios,Marca,BO TOTAL,Valor Facturado,Valor liberado,Valor Neto Pend,Valor Neto Retenido,Valor BO,control
0,2023-07-11,06:01:05,60036622,NaN,No relevante,NaN,Verificación de crédito no efectuada/Status no...,A,No tratado,MMARIGLIANO,2023-07-11,1010,BGH AR,VP,ARS,LH,H,Devoluciones,ZRE2,Dev. Comercial (0D),NaN,NaN,NOA,Noroeste,D010,Pago 60 días Fecha Factura,MERCADERIA DAÑADA,I04,INTE,Mario Marigliano,Remito 758-2602,1000244,CASTILLO SACIFIA,30,PNH048303,HORNO ELEC BGH BHE55M19N,7.796885e+12,100,NaN,-85165.29,NaN,UN,NaN,SP01,BG13,NaN,104001.0,NaN,1,X,2023-07-11,00:00:00,28388.43,ARS,NaN,NaN,NaN,NaN,NaT,D02,NaN,10010I10J,10I,ZREN,NaT,NaT,1,Cluster 1,NaN,NaN,RT,Retail Especializado,2023-07-11,A,No tratado,-3,UN,-3,UN,0,UN,0,UN,-3,UN,-3,UN,0,UN,0,UN,0,UN,0.0,NaN,0,UN,0,UN,0,UN,1000244,25 DE MAYO 320 PISO 2,14.0,8000190.0,TRANSPORTE LA SEVILLANITA SRL,12544.0,2023-07-11,NaN,NaN,NaN,NaT,NaN,NaT,NaN,SP,0,ARS,0,ARS,0,ARS,0,ARS,0,ARS,NaN,NaN,2023-07-11,Spot,HE,HE,HE,-,Liberado,Pendiente,Con Stock,Pendiente,1000244,1000244,1000244HESpot,0,7,2023,72023,1,1900,-85165.29,-3,Retail Especializado,OK,BGH,0,0.0,-85165.29,-85165.29,0.0,0.00,0.00
1,2023-07-11,06:01:05,60036622,NaN,No relevante,NaN,Verificación de crédito no efectuada/Status no...,A,No tratado,MMARIGLIANO,2023-07-11,1010,BGH AR,VP,ARS,LH,H,Devoluciones,ZRE2,Dev. Comercial (0D),NaN,NaN,NOA,Noroeste,D010,Pago 60 días Fecha Factura,MERCADERIA DAÑADA,I04,INTE,Mario Marigliano,Remito 758-2602,1000244,CASTILLO SACIFIA,10,PNH048317,HORNO ELEC BGH BHE64M20AN,7.796885e+12,100,NaN,-351059.50,NaN,UN,NaN,SP01,BG13,NaN,104001.0,NaN,1,X,2023-07-11,00:00:00,39006.61,ARS,NaN,NaN,NaN,NaN,NaT,D02,NaN,10010L10J,10L,ZREN,NaT,NaT,1,Cluster 1,NaN,NaN,RT,Retail Especializado,2023-07-11,A,No tratado,-9,UN,-9,UN,0,UN,0,UN,-9,UN,-9,UN,0,UN,0,UN,0,UN,0.0,NaN,0,UN,0,UN,0,UN,1000244,25 DE MAYO 320 PISO 2,14.0,8000190.0,TRANSPORTE LA SEVILLANITA SRL,12544.0,2023-07-11,NaN,NaN,NaN,NaT,NaN,NaT,NaN,SP,0,ARS,0,ARS,0,ARS,0,ARS,0,ARS,NaN,NaN,2023-07-11,Spot,HE,HE,HE,-,Liberado,Pendiente,Con Stock,Pendiente,1000244,1000244,1000244HESpot,0,7,2023,72023,1,1900,-351059.49,-9,Retail Especializado,OK,BGH,0,0.0,-351059.49,-351059.49,0.0,0.00,0.01
2,2

In [ ]:

# Especifica las columnas que vamos a conservar
columnas_deseadas = [
    'Creado el', 
    'Nombre Vendedor', 
    'Solicitante', 
    'Material', 
    'Número de material', 
    'ValorNeto', 
    'Precio Unitario', 
    'Fecha Fectura', 
    'Cant. Fact', 
    'Ctd.conf.pend', 
    'Cantidad en Back Order', 
    'Back Order por créditos', 
    'Cantidad en Back Order C o P', 
    'Tipo de venta', 
    'Línea', 
    'Línea OEM', 
    'AA', 
    'Retención ',  
    'Canal', 
    'Marca',
    'BO TOTAL',
    'Valor Facturado',
    'Valor liberado',
    'Valor Neto Pend',
    'Valor Neto Retenido',
    'Valor BO',
    'control',
    'Condición de pago - desc'
    
    
    
]

# Filtrar el DataFrame para quedarte solo con las columnas deseadas
df = df[columnas_deseadas]


In [10]:
df.to_excel('BDD_A_ESTUDIAR.xlsx',index=False)

In [7]:
df.columns

Index(['Creado el', 'Nombre Vendedor', 'Solicitante', 'Material',
       'Número de material', 'ValorNeto', 'Precio Unitario', 'Fecha Fectura',
       'Cant. Fact', 'Ctd.conf.pend', 'Cantidad en Back Order',
       'Back Order por créditos', 'Cantidad en Back Order C o P',
       'Tipo de venta', 'Línea', 'Línea OEM', 'AA', 'Retención ', 'Canal',
       'Marca', 'BO TOTAL', 'Valor Facturado', 'Valor liberado',
       'Valor Neto Pend', 'Valor Neto Retenido', 'Valor BO', 'control',
       'Condición de pago - desc'],
      dtype='object')

In [ ]:

# Configuración de la conexión
conexion_config = {
    'user': 'root',
    'password': 'Ayla2022',
    'host': 'localhost',
    'port': 3306
}

# Crear conexión para crear la base de datos 'BGH'
conexion = mysql.connector.connect(
    user=conexion_config['user'],
    password=conexion_config['password'],
    host=conexion_config['host'],
    port=conexion_config['port']
)

# Crear un cursor para ejecutar el SQL
cursor = conexion.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS BGH")

# Cerrar cursor y conexión
cursor.close()
conexion.close()

# Conectar a la base de datos 'BGH' recién creada
conexion_config['database'] = 'BGH'

# Crear el motor de SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{conexion_config['user']}:{conexion_config['password']}@{conexion_config['host']}:{conexion_config['port']}/{conexion_config['database']}")

# Limpiar los nombres de columnas eliminando espacios en blanco
df.columns = df.columns.str.strip()

nombre_tabla = 'ventas_diarias'
df.to_sql(nombre_tabla, con=engine, if_exists='append', index=False)

# Cerrar la conexión del engine
engine.dispose()


In [11]:
df_forecast = pd.read_excel("Bases Informe Diario de Ventas.xlsx",sheet_name='Forecast UN y $$')

In [12]:
df_forecast

,MARCA,CÓDIGOSAP,DESCRIPCIÓN,FORECAST SPOT,FORECAST PROG,FORECAST D2C,PVC SEPTIEMBRE (C/IVA) - CONTADO,Precio Prog Contado,Precio D2C,Fact Spot,Fact Prog,Fact D2C
0,HE,PNH048300,HE BGH 25L STD - BHE25M19N,1440.0,0.0,60.0,78948.204487,NaN,117084.142686,1.136854e+08,0.0,7.025049e+06
1,HE,PNH048301,HE BGH 30L STD - BHE30M19N,0.0,1000.0,0.0,86863.044826,96113.0225,137256.621578,0.000000e+00,96113022.5,0.000000e+00
2,HE,PNH048302,HE BGH 40L STD - BHE40M19N,800.0,0.0,100.0,96414.017903,NaN,31300.001700,7.713121e+07,0.0,3.130000e+06
3,HE,PNH048303,HE BGH 55L STD - BHE55M19N,380.0,0.0,120.0,116917.223353,NaN,173881.801431,4.442854e+07,0.0,2.086582e+07
4,HE,PNH048316,HE BGH 65L STD - BHE65M19,180.0,0.0,20.0,131093.034956,NaN,174062.175375,2.359675e+07,0.0,3.481244e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Celulares,NaN,NaN,NaN,NaN,500.0,166666.000000,NaN,246665.680000,0.000000e+00,NaN,1.233328e+08
76,Celulares,NaN,NaN,4000.0,NaN,NaN,199999.333333,NaN,295999.013333,7.999973e+08,NaN,0.000000e+00
77,Celulares,NaN,NaN,1000.0,NaN,NaN,199999.333333,NaN,295999.013333,1.999993e+08,NaN,0.000000e+00
78,Celulares,NaN,NaN,2800.0,NaN,1200.0,329999.333333,NaN,488399.013333,9.239981e+08,NaN,5.860788e+08


In [ ]:


# Configuración de la conexión
conexion_config = {
    'user': 'root',
    'password': 'Ayla2022',
    'host': 'localhost',
    'port': 3306
}

# Crear conexión para crear la base de datos 'BGH'
conexion = mysql.connector.connect(
    user=conexion_config['user'],
    password=conexion_config['password'],
    host=conexion_config['host'],
    port=conexion_config['port']
)

# Crear un cursor para ejecutar el SQL
cursor = conexion.cursor()


cursor.execute("CREATE DATABASE IF NOT EXISTS BGH")

# Cerrar cursor y conexión
cursor.close()
conexion.close()

# Conectar a la base de datos 'BGH' recién creada
conexion_config['database'] = 'BGH'

# Crear el motor de SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{conexion_config['user']}:{conexion_config['password']}@{conexion_config['host']}:{conexion_config['port']}/{conexion_config['database']}")

# Limpiar los nombres de columnas eliminando espacios en blanco
df_forecast.columns = df_forecast.columns.str.strip()

# Subir el DataFrame 'df_forecast' como la tabla 'forecast_octubre_2024'
nombre_tabla = 'forecast_octubre_2024'
df_forecast.to_sql(nombre_tabla, con=engine, if_exists='append', index=False)

# Cerrar la conexión del engine
engine.dispose()


In [16]:
df_acuerdos = pd.read_excel("Bases Informe Diario de Ventas.xlsx",sheet_name='acuerdos')

In [17]:
df_acuerdos

,Cliente,MARCA,Cliente Descripcion,Cliente Descripcion 2,Ac Comercial,Ac Logistico,EA,D24,TOTAL
0,INC-TV,TV,INC SOCIEDAD ANONIMA,TVINC SOCIEDAD ANONIMA,NaN,0.0400,0.0029,NaN,0.0029
1,INC-CMO,CMO,INC SOCIEDAD ANONIMA,CMOINC SOCIEDAD ANONIMA,NaN,0.0284,0.0029,NaN,0.0029
2,INC-HE,HE,INC SOCIEDAD ANONIMA,HEINC SOCIEDAD ANONIMA,NaN,0.0347,0.0029,NaN,0.0029
3,INC-AA,AAI,INC SOCIEDAD ANONIMA,AAIINC SOCIEDAD ANONIMA,NaN,0.0347,0.0029,NaN,0.0029
4,INC-TT,Aguasanitaria,INC SOCIEDAD ANONIMA,AguasanitariaINC SOCIEDAD ANONIMA,NaN,0.0347,0.0029,NaN,0.0029
5,INC-LV,Lavado,INC SOCIEDAD ANONIMA,LavadoINC SOCIEDAD ANONIMA,NaN,0.0347,0.0029,NaN,0.0029
6,INC-IFSA,Notebooks,INC SOCIEDAD ANONIMA,NotebooksINC SOCIEDAD ANONIMA,NaN,0.0338,0.0029,NaN,0.0029
7,JUMBO-TV,TV,CENCOSUD S.A.,TVCENCOSUD S.A.,NaN,0.0700,NaN,0.07,0.0700
8,JUMBO-CMO,CMO,CENCOSUD S.A.,CMOCENCOSUD S.A.,NaN,0.0700,0.0750,0.07,0.1450
9,JUMBO-HE,HE,CENCOSUD S.A.,HECENCOSUD S.A.,NaN,0.0700,NaN,0.07,0.0700


In [ ]:

# Configuración de la conexión
conexion_config = {
    'user': 'root',
    'password': 'Ayla2022',
    'host': 'localhost',
    'port': 3306
}

# Crear conexión para crear la base de datos 'BGH'
conexion = mysql.connector.connect(
    user=conexion_config['user'],
    password=conexion_config['password'],
    host=conexion_config['host'],
    port=conexion_config['port']
)

# Crear un cursor para ejecutar el SQL
cursor = conexion.cursor()

# Crear la base de datos 'BGH' si no existe
cursor.execute("CREATE DATABASE IF NOT EXISTS BGH")

# Cerrar cursor y conexión
cursor.close()
conexion.close()

# Conectar a la base de datos 'BGH' recién creada
conexion_config['database'] = 'BGH'

# Crear el motor de SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{conexion_config['user']}:{conexion_config['password']}@{conexion_config['host']}:{conexion_config['port']}/{conexion_config['database']}")

# Limpiar los nombres de columnas eliminando espacios en blanco
df_acuerdos.columns = df_acuerdos.columns.str.strip()

# Subir el DataFrame 'df_acuerdos' como la tabla 'acuerdos_clientes'
nombre_tabla = 'acuerdos_clientes'
df_acuerdos.to_sql(nombre_tabla, con=engine, if_exists='append', index=False)

# Cerrar la conexión del engine
engine.dispose()


In [18]:
df_precios = pd.read_excel("Info PVPs (precios).xlsx", sheet_name='download - 2024-04-04T123146.19')

In [ ]:

# Configuración de la conexión
conexion_config = {
    'user': 'root',
    'password': 'Ayla2022',
    'host': 'localhost',
    'port': 3306
}

# Crear conexión para crear la base de datos 'BGH'
conexion = mysql.connector.connect(
    user=conexion_config['user'],
    password=conexion_config['password'],
    host=conexion_config['host'],
    port=conexion_config['port']
)

# Crear un cursor para ejecutar el SQL
cursor = conexion.cursor()

# Crear la base de datos 'BGH' si no existe
cursor.execute("CREATE DATABASE IF NOT EXISTS BGH")

# Cerrar cursor y conexión
cursor.close()
conexion.close()

# Conectar a la base de datos 'BGH' recién creada
conexion_config['database'] = 'BGH'

# Crear el motor de SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{conexion_config['user']}:{conexion_config['password']}@{conexion_config['host']}:{conexion_config['port']}/{conexion_config['database']}")

# Limpiar los nombres de columnas eliminando espacios en blanco
df_precios.columns = df_precios.columns.str.strip()

# Subir el DataFrame 'df_precios' como la tabla 'precios_pvp'
nombre_tabla = 'precios_pvp'
df_precios.to_sql(nombre_tabla, con=engine, if_exists='append', index=False)

# Cerrar la conexión del engine
engine.dispose()


In [7]:
df_tasas = pd.read_excel("Base Historica.xlsx", sheet_name='Tasas')

In [8]:
df_tasas

,Dias,Dias2,Tasa de porcentaje,Descripcion condicion de pago
0,7 días,7,0.007096,BGH Pago 7 días Fecha Factura
1,60 días,60,0.060822,BGH Pago 60 días Fecha Factura
2,Contado,1,0.010000,BGH Ecommerce CONTADO
3,15 días,15,0.015205,BGH Pago 15 días Fecha Factura
4,Contado,1,0.010000,BGH Pago Inmediato Fecha Factura
5,30 días,30,0.030411,BGH Pago 30 días Fecha Factura
6,5 días,5,0.005068,BGH Pago 5 días Fecha Factura
7,45 días,45,0.045616,BGH Pago 45 días Fecha Factura
8,10 días,10,0.010137,BGH Pago 10 días Fecha Factura
9,20 días,20,0.020274,BGH Pago 20 días Fecha Factura


In [ ]:

# Conectar a la base de datos BGH
conexion_config = {
    'user': 'root',
    'password': 'Ayla2022',
    'host': 'localhost',
    'port': 3306,
    'database': 'BGH'
}

# Crear el motor de SQLAlchemy
engine = create_engine(f"mysql+mysqlconnector://{conexion_config['user']}:{conexion_config['password']}@{conexion_config['host']}:{conexion_config['port']}/{conexion_config['database']}")

# Función para subir un DataFrame en partes
def subir_df_por_partes(df, nombre_tabla, engine, partes=5):
    try:
        # Limpiar los nombres de columnas eliminando espacios en blanco
        df.columns = df.columns.str.strip()

        # Dividir el DataFrame en 'partes' partes
        dfs = np.array_split(df, partes)

        # Subir cada parte del DataFrame
        for i, df_part in enumerate(dfs):
            print(f"Subiendo parte {i+1} de {partes}...")
            df_part.to_sql(nombre_tabla, con=engine, if_exists='append', index=False)
        
        print(f"Tabla '{nombre_tabla}' subida correctamente en {partes} partes.")

    except Exception as e:
        print(f"Error al subir la tabla '{nombre_tabla}': {e}")
        # Intentar rollback manual
        try:
            engine.raw_connection().rollback()
            print("Rollback realizado manualmente.")
        except Exception as rollback_error:
            print(f"Error al intentar hacer rollback: {rollback_error}")

        engine.dispose()
        return False

    return True

# Dividir el DataFrame 'df_tasas' en 5 partes y subirlo
nombre_tabla = 'tasas'
subir_df_por_partes(df_tasas, nombre_tabla, engine, partes=5)

# Cerrar la conexión del engine
engine.dispose()


c:\Users\sosaj\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Subiendo parte 1 de 5...
Subiendo parte 2 de 5...
Subiendo parte 3 de 5...
Subiendo parte 4 de 5...
Subiendo parte 5 de 5...
Tabla 'tasas' subida correctamente en 5 partes.


Esto es un vistaso de las bases de datos ya cargadas en Mysql

![](Mysql.PNG)
